# Teleco Customer Churn Prediction

Definiton of Churn:

Telecommunication customer churn refers to an event in which customers of a telecommunication service provider decide to discontinue their services and switch to another provider or terminate their services entirely. Churn is an important metric for telecom companies because it has a direct impact on revenue, customer base, and overall profitability. High churn rates can indicate customer dissatisfaction, poor service quality, or intense industry competition. 

Objective:

Our objective is to make a neural network model which can predict behavior to retain customers beforehand with the available details of the customer and know whether the customer will continue the services or not

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
from sklearn import model_selection
import matplotlib.pyplot as plt

# Getting the data

The dataset is available publicly on [kaggle](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)  
The dataset is downloaded locally and used in the notebook in form of Pandas DataFrame

In [3]:
df=pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [4]:
# checking the first 5 rows of the dataframe
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


# Exploratory Data Analysis and Preprocessing

Performing EDA on the dataset to remove any discrepancies and abnormality in data set

In [5]:
# checking the information of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
# checking for any null or NaN values
print('Total number of NaN values: ',df.isna().sum().sum())
print('Total number of null values: ',df.isnull().sum().sum())

Total number of NaN values:  0
Total number of null values:  0


In [6]:
# reducing few columns which are not necessary for further use
df.drop(['customerID','Dependents','MultipleLines','PaperlessBilling','OnlineSecurity','PaymentMethod','StreamingTV'],
        axis=1,inplace=True)

In [8]:
# columns left after removal
print(df.columns)

Index(['gender', 'SeniorCitizen', 'Partner', 'tenure', 'PhoneService',
       'InternetService', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingMovies', 'Contract', 'MonthlyCharges', 'TotalCharges',
       'Churn'],
      dtype='object')


We need to preprocess the data as TensorFlow ANN model do not use Text input in the Dense Layers  
With the help of **_sklearn.preprocessing_** module we can encode our text data into numerical values

In [9]:
# Using the LabelEncoder function to encode the columns of our dataset which are of Object Data Type
label_encoder = preprocessing.LabelEncoder()
def encoding(df):
    a=df.dtypes.to_dict()
    for i in a:
        if a[i]=='object':
            df[i]=label_encoder.fit_transform(df[i])
    return df
df=encoding(df)

In [10]:
# checking the first 5 rows of dataframe after encoding
df.head()

,gender,SeniorCitizen,Partner,tenure,PhoneService,InternetService,OnlineBackup,DeviceProtection,TechSupport,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,0,0,1,1,0,0,2,0,0,0,0,29.85,2505,0
1,1,0,0,34,1,0,0,2,0,0,1,56.95,1466,0
2,1,0,0,2,1,0,2,0,0,0,0,53.85,157,1
3,1,0,0,45,0,0,0,2,2,0,1,42.30,1400,0
4,0,0,0,2,1,1,0,0,0,0,0,70.70,925,1


In [11]:
# The data type of columns which were object earlier is now changed to integer type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   int32  
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   int32  
 3   tenure            7043 non-null   int64  
 4   PhoneService      7043 non-null   int32  
 5   InternetService   7043 non-null   int32  
 6   OnlineBackup      7043 non-null   int32  
 7   DeviceProtection  7043 non-null   int32  
 8   TechSupport       7043 non-null   int32  
 9   StreamingMovies   7043 non-null   int32  
 10  Contract          7043 non-null   int32  
 11  MonthlyCharges    7043 non-null   float64
 12  TotalCharges      7043 non-null   int32  
 13  Churn             7043 non-null   int32  
dtypes: float64(1), int32(11), int64(2)
memory usage: 467.8 KB


# Model selection

Defining X and Y which will be our independent and dependent variable respectively  
We will split our dataset into two parts with help pf **_sklearn.model_selection_**
1. Train Set: It contains 85% of the data i.e. 5986 data points
1. Test Set: It contains 15% of the data i.e. 1057 data points  


We will then train various TensorFlow Logistic Regression models and compare them to find the best model out of it

In [12]:
y=df.Churn.to_numpy().reshape(-1,1)
x=df.drop('Churn',axis=1).to_numpy()

In [14]:
# using the train test split method in model_selection to split the data set
x_train,x_test,y_train,y_test=model_selection.train_test_split(x,y,test_size=0.15,random_state=1)

Model 1 Description

* The model contains 2 hidden layers and one output layer


    1. Layer 1 contains 10 nodes and activation function as ReLU
    1. Layer 2 contains 5 nodes and activation function as ReLU
    1. Output Layer contains 1 node and activation function as Sigmoid 
    
    
* Model uses Adam optimizer with learing rate of 0.01
* Model uses Binary_CrossEntropy loss to calculate the loss at every step and metrics is accuracy
* Model is ran over 100 epochs

In [17]:
model1=keras.Sequential([
    keras.layers.Dense(10,activation='relu'),
    keras.layers.Dense(5,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])
model1.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
fit=model1.fit(x_train,y_train,epochs=100)

Epoch 1/100
188/188 [==============================] - 3s 4ms/step - loss: 67.2337 - accuracy: 0.5416
Epoch 2/100
188/188 [==============================] - 1s 4ms/step - loss: 0.9102 - accuracy: 0.6614
Epoch 3/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5253 - accuracy: 0.7491
Epoch 4/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5389 - accuracy: 0.7516
Epoch 5/100
188/188 [==============================] - 1s 3ms/step - loss: 0.5826 - accuracy: 0.7446
Epoch 6/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5361 - accuracy: 0.7551
Epoch 7/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5398 - accuracy: 0.7578
Epoch 8/100
188/188 [==============================] - 1s 4ms/step - loss: 0.6019 - accuracy: 0.7489
Epoch 9/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5419 - accuracy: 0.7566
Epoch 10/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5410 - accur

188/188 [==============================] - 1s 4ms/step - loss: 0.5704 - accuracy: 0.7501
Epoch 82/100
188/188 [==============================] - 1s 3ms/step - loss: 0.5517 - accuracy: 0.7556
Epoch 83/100
188/188 [==============================] - 1s 4ms/step - loss: 0.7300 - accuracy: 0.7354
Epoch 84/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5654 - accuracy: 0.7579
Epoch 85/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5093 - accuracy: 0.7624
Epoch 86/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5408 - accuracy: 0.7539
Epoch 87/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5615 - accuracy: 0.7631
Epoch 88/100
188/188 [==============================] - 1s 3ms/step - loss: 0.5638 - accuracy: 0.7482
Epoch 89/100
188/188 [==============================] - 1s 3ms/step - loss: 0.5525 - accuracy: 0.7514
Epoch 90/100
188/188 [==============================] - 1s 3ms/step - loss: 0.6775 - accuracy: 

In [20]:
# predicting Y from our first model
y_pred=model1.predict(x_test)

34/34 [==============================] - 0s 4ms/step


Preparing the confusion matrix from **_Sklearn.metrics_** to check for the accuracy

In [22]:
from sklearn.metrics import confusion_matrix
# rounding the off the values to make Y_pred only 0 or 1
y_pred=[np.round(x) for x in y_pred]
cn=confusion_matrix(y_pred,y_test)
print(cn)

[[628  70]
 [170 189]]


In [24]:
print('Total number of false negatives: ',cn[0][1])
print('Total number of false positives: ',cn[1][0])
print('Model Accuracy on test set is: ',(cn[0][0]+cn[1][1])/(cn[0][0]+cn[0][1]+cn[1][0]+cn[1][1]))

Total number of false negatives:  70
Total number of false positives:  170
Model Accuracy on test set is:  0.7729422894985809


Model 2 Description

* The model contains 3 hidden layers and one output layer


    1. Layer 1 contains 15 nodes and activation function as ReLU
    1. Layer 2 contains 10 nodes and activation function as ReLU
    1. Layer 3 contains 5 nodes and activation function as ReLU
    1. Output Layer contains 1 node and activation function as Sigmoid 
    
    
* Model uses Adam optimizer with learing rate of 0.001
* Model uses Binary_CrossEntropy loss to calculate the loss at every step and metrics is accuracy
* Model is ran over 100 epochs

In [50]:
model2=keras.Sequential([
    keras.layers.Dense(15,activation='relu'),
    keras.layers.Dense(10,activation='relu'),
    keras.layers.Dense(5,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])
model2.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
fit=model2.fit(x_train,y_train,epochs=100)

Epoch 1/100
188/188 [==============================] - 3s 4ms/step - loss: 16.9379 - accuracy: 0.6617
Epoch 2/100
188/188 [==============================] - 1s 4ms/step - loss: 0.9633 - accuracy: 0.7444
Epoch 3/100
188/188 [==============================] - 1s 4ms/step - loss: 0.7264 - accuracy: 0.7414
Epoch 4/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5884 - accuracy: 0.7504
Epoch 5/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5884 - accuracy: 0.7421
Epoch 6/100
188/188 [==============================] - 1s 4ms/step - loss: 0.7044 - accuracy: 0.7347
Epoch 7/100
188/188 [==============================] - 1s 4ms/step - loss: 0.7170 - accuracy: 0.7250
Epoch 8/100
188/188 [==============================] - 1s 4ms/step - loss: 0.6336 - accuracy: 0.7454
Epoch 9/100
188/188 [==============================] - 1s 4ms/step - loss: 0.6025 - accuracy: 0.7462
Epoch 10/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5704 - accur

188/188 [==============================] - 1s 4ms/step - loss: 0.4741 - accuracy: 0.7700
Epoch 82/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5139 - accuracy: 0.7539
Epoch 83/100
188/188 [==============================] - 1s 4ms/step - loss: 0.4713 - accuracy: 0.7703
Epoch 84/100
188/188 [==============================] - 1s 4ms/step - loss: 0.4636 - accuracy: 0.7765
Epoch 85/100
188/188 [==============================] - 1s 3ms/step - loss: 0.4757 - accuracy: 0.7713
Epoch 86/100
188/188 [==============================] - 1s 4ms/step - loss: 0.4580 - accuracy: 0.7773
Epoch 87/100
188/188 [==============================] - 1s 4ms/step - loss: 0.5210 - accuracy: 0.7559
Epoch 88/100
188/188 [==============================] - 1s 4ms/step - loss: 0.4659 - accuracy: 0.7705
Epoch 89/100
188/188 [==============================] - 1s 4ms/step - loss: 0.4767 - accuracy: 0.7756
Epoch 90/100
188/188 [==============================] - 1s 4ms/step - loss: 0.4677 - accuracy: 

In [51]:
# predicting Y from new model
y_pred=model2.predict(x_test)

34/34 [==============================] - 0s 3ms/step


In [52]:
from sklearn.metrics import confusion_matrix
# rounding the off the values to make Y_pred only 0 or 1
y_pred=[np.round(x) for x in y_pred]
cn=confusion_matrix(y_pred,y_test)
print('confusion matrix: ')
print(cn)

confusion matrix: 
[[769 218]
 [ 29  41]]


In [53]:
print('Total number of false negatives: ',cn[0][1])
print('Total number of false positives: ',cn[1][0])
print('Model Accuracy on test set is: ',(cn[0][0]+cn[1][1])/(cn[0][0]+cn[0][1]+cn[1][0]+cn[1][1]))

Total number of false negatives:  218
Total number of false positives:  29
Model Accuracy on test set is:  0.7663197729422895


Model 3 Description

* The model contains 6 hidden layers and one output layer


    1. Layer 1 contains 40 nodes and activation function as linear
    1. Layer 2 contains 30 nodes and activation function as linear
    1. Layer 3 contains 20 nodes and activation function as ReLU
    1. Layer 4 contains 15 nodes and activation function as ReLU
    1. Layer 5 contains 10 nodes and activation function as ReLU
    1. Layer 6 contains 5 nodes and activation function as ReLU
    1. Output Layer contains 1 node and activation function as Sigmoid 
    
    
* Model uses Adam optimizer with learing rate of 0.001
* Model uses Binary_CrossEntropy loss to calculate the loss at every step and metrics is accuracy
* Model is ran over 1000 epochs

In [38]:
model=keras.Sequential([
    keras.layers.Dense(40,activation='linear'),
    keras.layers.Dense(30,activation='linear'),
    keras.layers.Dense(20,activation='relu'),
    keras.layers.Dense(15,activation='relu'),
    keras.layers.Dense(10,activation='relu'),
    keras.layers.Dense(5,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])
model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [39]:
fit1=model.fit(x_train,y_train,epochs=1000)

Epoch 1/1000
188/188 [==============================] - 5s 5ms/step - loss: 7.5238 - accuracy: 0.6640
Epoch 2/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.8892 - accuracy: 0.6980
Epoch 3/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.9132 - accuracy: 0.7080
Epoch 4/1000
188/188 [==============================] - 1s 4ms/step - loss: 1.1592 - accuracy: 0.7125
Epoch 5/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.6338 - accuracy: 0.7447
Epoch 6/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.5902 - accuracy: 0.7421
Epoch 7/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.6203 - accuracy: 0.7454
Epoch 8/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.6186 - accuracy: 0.7402
Epoch 9/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.7156 - accuracy: 0.7230
Epoch 10/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.590

188/188 [==============================] - 1s 4ms/step - loss: 0.5805 - accuracy: 0.7315
Epoch 81/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.5813 - accuracy: 0.7319
Epoch 82/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.5807 - accuracy: 0.7317
Epoch 83/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.5813 - accuracy: 0.7312
Epoch 84/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.5810 - accuracy: 0.7317
Epoch 85/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.5811 - accuracy: 0.7319
Epoch 86/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.5804 - accuracy: 0.7327
Epoch 87/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.6031 - accuracy: 0.7297
Epoch 88/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.5803 - accuracy: 0.7315
Epoch 89/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.5799 - a

188/188 [==============================] - 1s 4ms/step - loss: 0.4621 - accuracy: 0.7731
Epoch 160/1000
188/188 [==============================] - 1s 5ms/step - loss: 0.4495 - accuracy: 0.7837
Epoch 161/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4582 - accuracy: 0.7750
Epoch 162/1000
188/188 [==============================] - 1s 5ms/step - loss: 0.4833 - accuracy: 0.7631
Epoch 163/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4568 - accuracy: 0.7792
Epoch 164/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4487 - accuracy: 0.7880
Epoch 165/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4560 - accuracy: 0.7805
Epoch 166/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4590 - accuracy: 0.7715
Epoch 167/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4507 - accuracy: 0.7790
Epoch 168/1000
188/188 [==============================] - 1s 4ms/step - loss: 0

188/188 [==============================] - 1s 4ms/step - loss: 0.4369 - accuracy: 0.7868
Epoch 238/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4442 - accuracy: 0.7858
Epoch 239/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4393 - accuracy: 0.7855
Epoch 240/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4378 - accuracy: 0.7882
Epoch 241/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4491 - accuracy: 0.7822
Epoch 242/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4424 - accuracy: 0.7857
Epoch 243/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4400 - accuracy: 0.7860
Epoch 244/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4392 - accuracy: 0.7855
Epoch 245/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4444 - accuracy: 0.7830
Epoch 246/1000
188/188 [==============================] - 1s 4ms/step - loss: 0

188/188 [==============================] - 1s 4ms/step - loss: 0.4375 - accuracy: 0.7907
Epoch 316/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4381 - accuracy: 0.7865
Epoch 317/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4343 - accuracy: 0.7907
Epoch 318/1000
188/188 [==============================] - 1s 5ms/step - loss: 0.4369 - accuracy: 0.7908
Epoch 319/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4367 - accuracy: 0.7920
Epoch 320/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4488 - accuracy: 0.7815
Epoch 321/1000
188/188 [==============================] - 1s 5ms/step - loss: 0.4439 - accuracy: 0.7863
Epoch 322/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4341 - accuracy: 0.7905
Epoch 323/1000
188/188 [==============================] - 1s 5ms/step - loss: 0.4426 - accuracy: 0.7900
Epoch 324/1000
188/188 [==============================] - 1s 5ms/step - loss: 0

...

188/188 [==============================] - 1s 4ms/step - loss: 0.4267 - accuracy: 0.7917
Epoch 628/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4292 - accuracy: 0.7955
Epoch 629/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4266 - accuracy: 0.7952
Epoch 630/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4430 - accuracy: 0.7815
Epoch 631/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4308 - accuracy: 0.7920
Epoch 632/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4354 - accuracy: 0.7927
Epoch 633/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4310 - accuracy: 0.7939
Epoch 634/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4325 - accuracy: 0.7872
Epoch 635/1000
188/188 [==============================] - 1s 5ms/step - loss: 0.4266 - accuracy: 0.7918
Epoch 636/1000
188/188 [==============================] - 1s 5ms/step - loss: 0

188/188 [==============================] - 1s 4ms/step - loss: 0.4349 - accuracy: 0.7893
Epoch 706/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4305 - accuracy: 0.7905
Epoch 707/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4346 - accuracy: 0.7880
Epoch 708/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4293 - accuracy: 0.7898
Epoch 709/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4330 - accuracy: 0.7920
Epoch 710/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4285 - accuracy: 0.7915
Epoch 711/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4292 - accuracy: 0.7927
Epoch 712/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4354 - accuracy: 0.7905
Epoch 713/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4375 - accuracy: 0.7903
Epoch 714/1000
188/188 [==============================] - 1s 4ms/step - loss: 0

188/188 [==============================] - 1s 4ms/step - loss: 0.4278 - accuracy: 0.7944
Epoch 784/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4269 - accuracy: 0.7944
Epoch 785/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4331 - accuracy: 0.7928
Epoch 786/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4391 - accuracy: 0.7935
Epoch 787/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4318 - accuracy: 0.7937
Epoch 788/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4301 - accuracy: 0.7937
Epoch 789/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4293 - accuracy: 0.7928
Epoch 790/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4307 - accuracy: 0.7979
Epoch 791/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4278 - accuracy: 0.7962
Epoch 792/1000
188/188 [==============================] - 1s 4ms/step - loss: 0

188/188 [==============================] - 1s 4ms/step - loss: 0.4229 - accuracy: 0.7979
Epoch 862/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4230 - accuracy: 0.7967
Epoch 863/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4257 - accuracy: 0.7937
Epoch 864/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4213 - accuracy: 0.7969
Epoch 865/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4251 - accuracy: 0.7955
Epoch 866/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4274 - accuracy: 0.7957
Epoch 867/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4254 - accuracy: 0.7927
Epoch 868/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4237 - accuracy: 0.7974
Epoch 869/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4246 - accuracy: 0.7935
Epoch 870/1000
188/188 [==============================] - 1s 4ms/step - loss: 0

188/188 [==============================] - 1s 4ms/step - loss: 0.4245 - accuracy: 0.7979
Epoch 940/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4242 - accuracy: 0.7974
Epoch 941/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4238 - accuracy: 0.7934
Epoch 942/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4232 - accuracy: 0.7950
Epoch 943/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4236 - accuracy: 0.8004
Epoch 944/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4225 - accuracy: 0.7949
Epoch 945/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4245 - accuracy: 0.7962
Epoch 946/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4250 - accuracy: 0.7937
Epoch 947/1000
188/188 [==============================] - 1s 4ms/step - loss: 0.4243 - accuracy: 0.7942
Epoch 948/1000
188/188 [==============================] - 1s 4ms/step - loss: 0

In [54]:
# predicting Y from new model
y_pred=model.predict(x_test)

34/34 [==============================] - 0s 3ms/step


In [55]:
from sklearn.metrics import confusion_matrix
# rounding the off the values to make Y_pred only 0 or 1
y_pred=[np.round(x) for x in y_pred]
cn=confusion_matrix(y_pred,y_test)
print('confusion matrix: ')
print(cn)

confusion matrix: 
[[746 159]
 [ 52 100]]


In [57]:
print('Total number of false negatives: ',cn[0][1])
print('Total number of false positives: ',cn[1][0])
print('Model Accuracy on test set is: ',(cn[0][0]+cn[1][1])/(cn[0][0]+cn[0][1]+cn[1][0]+cn[1][1]))

Total number of false negatives:  159
Total number of false positives:  52
Model Accuracy on test set is:  0.8003784295175024


In [23]:
# saving the model to deploy it using Streamlit
model.save('mymodel3')

INFO:tensorflow:Assets written to: mymodel3\assets


INFO:tensorflow:Assets written to: mymodel3\assets


In [24]:
# Checking whether the saved model is working or not
loaded_model=tf.keras.models.load_model('mymodel3')

In [25]:
yP=loaded_model.predict(x_test)

34/34 [==============================] - 0s 2ms/step


# Conclusion

The last model "Model3" was the best out of the three models having the accuracy of 80.03%  
The same model will be used in the Streamlit App to predict customer churn  
Link: [Aditya-Customer-Churn-Prediction](https://aditya-customer-churn-prediction.streamlit.app/)